## Gate cutting to reduce SWAP overhead

In [1]:
import numpy as np
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import PauliList

Specify a circuit and some observables

In [2]:
circuit = EfficientSU2(num_qubits=4, entanglement="circular").decompose()
circuit.assign_parameters([0.4] * len(circuit.parameters), inplace=True)
observables = PauliList(["ZZII", "IZZI", "IIZZ", "XIXI", "ZIZZ", "IXIX"])

Cut the long-distant gates by using ``cut_gates`` and specifying their indices. Generate the subexperiments and their weights using ``generate_cutting_experiments``.

In [3]:
from circuit_knitting.cutting import cut_gates, generate_cutting_experiments

# Find the indices of the distant gates
cut_indices = [
    i
    for i, instruction in enumerate(circuit.data)
    if {circuit.find_bit(q)[0] for q in instruction.qubits} == {0, 3}
]

# Cut the long-distant gates and generate the subexperiments
qpd_circuit, bases = cut_gates(circuit, cut_indices)
subexperiments, weights = generate_cutting_experiments(qpd_circuit, np.inf, observables)

Run the subexperiments for each partition using the Qiskit Aer Sampler primitive.

In [4]:
from qiskit_aer.primitives import Sampler

# Set up Qiskit Aer Sampler primitive.
sampler = Sampler(run_options={"shots": 2**12})

# Retrieve the results from each subexperiment
results = sampler.run(subexperiments).result()

To run using Qiskit Runtime Sampler primitives, replace the code above with this commented block.

In [5]:
# from qiskit_ibm_runtime import Session, Options, Sampler

# with Session(backend="<BACKEND_NAME>") as session:
#   options = Options()
#   options.execution.shots = 2**12
#   sampler = Sampler(options=options)
#   results = sampler.run(subexperiments).result()

#   session.close()

# for i in range(len(subexperiments)):
#    results.metadata[i]['num_qpd_bits'] = len(partitioned_problem.subexperiments[i].cregs[0])

Reconstruct the full expectation value, given the results of cutting

In [6]:
from circuit_knitting.cutting import reconstruct_expectation_values

# Include the number of bits used for cutting measurements in the results
for i in range(len(subexperiments)):
    results.metadata[i]["num_qpd_bits"] = len(subexperiments[i].cregs[0])

simulated_expvals = reconstruct_expectation_values(
    observables,
    weights,
    results,
)
simulated_expvals

[0.5514526367187484,
 0.5217895507812491,
 0.416992187499999,
 -0.23541259765624933,
 0.21868896484374958,
 -0.20617675781249947]